In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import GaussianNB
#  importando los modulos confusion_matrix, roc_curve, auc, accuracy_score de sklearn.metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, classification_report
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_predict.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Var_Rpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Trasaccionales.xlsx
/kaggle/input/datatn-bancolombia-2019/Trminos y Condiciones Datatn 2019.pdf
/kaggle/input/datatn-bancolombia-2019/presentacin Dataton BC 2019.pdf
/kaggle/input/datatn-bancolombia-2019/DT19_maestro_cdgtrn_cdgrpta.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos maestro_cdgtrn_cdgrpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train.csv


In [4]:
dataY = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train_lite.csv')


In [5]:
dataY.head()

,id,f_analisis,var_rpta,segmento
0,1,201803,0,4
1,2,201604,0,0
2,3,201608,0,5
3,4,201706,0,4
4,5,201703,0,4


## lets try a very simple model

In [16]:
colnames = data.columns.values.tolist()
predictors = [colnames[3]]
targetName = colnames[2]

## Very simple experimental setup

In [26]:
# Create a new array with the added features: features_two

features = dataY[predictors]
target = dataY[targetName]
# Split the data into train and test
trainX, testX, trainY, testY = train_test_split(features, target, test_size = 0.2, stratify = target )
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(30400, 1) (30400,)
(7600, 1) (7600,)


In [37]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# NN is sensitive to data scale. We must normilize
scaler = StandardScaler()  
trainXX = trainX.copy()
testXX = testX.copy()
# Don't cheat - fit only on training data
scaler.fit(trainX)  
trainXX = scaler.transform(trainXX)  
# apply same transformation to test data
testXX = scaler.transform(testXX)  

model3 = MLPClassifier(solver='lbfgs', alpha=1e-3, activation = 'logistic', max_iter=5000,
                     hidden_layer_sizes = [10, 5, 3], verbose = True)

model3.fit(trainXX, trainY)                         

#Print the score on the train data
print("On training")
print(model3.score(trainXX, trainY))
print(confusion_matrix(model3.predict(trainXX), trainY))

#Print the score on the test data
print("\nOn test")
print(model3.score(testXX, testY))
print(confusion_matrix(model3.predict(testXX), testY))

print("ROC: ")
print(roc_auc_score(testY, model3.predict(testXX)))

On training
0.9266776315789473
[[28171  2229]
 [    0     0]]

On test
0.9267105263157894
[[7043  557]
 [   0    0]]
ROC: 
0.5


## Make a new prediction and save it in the output file

In [54]:
data2 = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv')

#Make a prediction
data2['probabilidad'] =  model3.predict(data2[['segmento']])

data_final = data2[['id', 'probabilidad']]

# Save to a file. Find it in /kaggle/output/working/
data_final.to_csv("DT19_IDs_predict.csv", index=False)


In [49]:
 model3.predict(data2[['segmento']])

array([0, 0, 0, ..., 0, 0, 0])